In [2]:
import os
from data_management import DataSource
from file_management import (SiteHierarchies, IndexFile,  FormulaFile)
from formula_tables import (FormulaTable, FormulaTableType,
                            FormulaTableTypeSimple, FormulaTableTypeCalculus,
                            FormulaTableTypeFinancial)
from content import Formulas, Definitions
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies)
formulas_by_year = Formulas(data_source.formulas_by_year)
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)
formulas_cumulative.is_cumulative = True

In [3]:
definitions = Definitions(data_source.definitions_by_year)

definitions.to_dataframe()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Definition
0,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,annuities
1,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,sequence
2,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,series
3,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,arithmetic sequence and series
4,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,common difference
5,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,geometric sequence and series
6,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,common ratio
7,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,present value
8,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,future value
